# Volatility Indices Calculation

This notebook explains how the module *vxbt_calc* calculates the XVBT, AXVBT and GXVBT indices using data from Deribit.

In [2]:
import calendar
import numpy as np
import openapi_client as dbitApi
import pandas as pd

from datetime import datetime

### Utility functions for time calculations, Deribit API and dataframe formatting

In [201]:
def format_datetime_to_expiry(date):
    return datetime.strftime(date, '%-d%b%y').upper()

def get_near_next_terms(now):
    c = calendar.Calendar(firstweekday=calendar.MONDAY)
    
    this_month_cal = c.monthdatescalendar(now.year, now.month)
    this_fridays = [datetime(day.year, day.month, day.day, 8, 0, 0) 
                    for week in this_month_cal for day in week 
                    if day.weekday() == calendar.FRIDAY and day.month == now.month 
                    and datetime(day.year, day.month, day.day, 8, 0, 0) >= now]
    
    next_year = now.year if now.month < 12 else now.year + 1
    next_month = now.month + 1 if now.month < 12 else 1
    
    next_month_cal = c.monthdatescalendar(next_year, next_month)
    next_fridays = [datetime(day.year, day.month, day.day, 8, 0, 0) 
                    for week in next_month_cal for day in week 
                    if day.weekday() == calendar.FRIDAY and day.month == next_month 
                    and datetime(day.year, day.month, day.day, 8, 0, 0) >= now]
    
    fridays = this_fridays + next_fridays
    
    near_term, next_term = fridays[0], fridays[1]
        
    return (format_datetime_to_expiry(near_term), format_datetime_to_expiry(next_term), near_term, next_term)

def get_index(currency='BTC'):
    try:
        index_result = api.public_get_index_get(currency)['result'][currency]
        return index_result
    except dbitApi.exceptions.ApiException as e:
        print(e)
        #logger.exception('Exception when calling MarketDataApi->public_get_instruments_get!')
        exit()

def get_instruments_with_expiry(expiry, currency='BTC', kind='option', expired='false'):
    try:
        instrument_result = api.public_get_instruments_get(currency, kind=kind, expired=expired)['result']
        return [instrument['instrument_name'] for instrument in instrument_result if expiry in instrument['instrument_name']]
    except dbitApi.exceptions.ApiException as e:
        print(e)
        #logger.exception('Exception when calling MarketDataApi->public_get_instruments_get!')
        exit()

def get_ticker(instrument):
    try:
        instrument_result = api.public_ticker_get(instrument)['result']
        return instrument_result
    except dbitApi.exceptions.ApiException as e:
        print(e)
        #logger.exception('Exception when calling MarketDataApi->public_get_instruments_get!')
        exit()

def get_bids_asks(near_list, next_list):
    near_calls = dict()
    near_puts = dict()
    next_calls = dict()
    next_puts = dict()

    for instrument in near_list:
        data = get_ticker(instrument)
        best_bid, best_ask = data['best_bid_price'], data['best_ask_price']
        strike, cp = int(instrument.split('-')[2]), instrument.split('-')[3]

        if cp == 'C':
            near_calls[strike] = {'best_bid': best_bid, 'best_ask': best_ask}
        elif cp == 'P':
            near_puts[strike] = {'best_bid': best_bid, 'best_ask': best_ask}
        else:
            print(f'Error {instrument}')

    for instrument in next_list:
        data = get_ticker(instrument)
        best_bid, best_ask = data['best_bid_price'], data['best_ask_price']
        strike, cp = int(instrument.split('-')[2]), instrument.split('-')[3]

        if cp == 'C':
            next_calls[strike] = {'best_bid': best_bid, 'best_ask': best_ask}
        elif cp == 'P':
            next_puts[strike] = {'best_bid': best_bid, 'best_ask': best_ask}
        else:
            print(f'Error {instrument}')

    near_calls_df = pd.DataFrame.from_dict(near_calls, orient='index').sort_index().replace(0, np.nan)
    near_puts_df = pd.DataFrame.from_dict(near_puts, orient='index').sort_index().replace(0, np.nan)
    next_calls_df = pd.DataFrame.from_dict(next_calls, orient='index').sort_index().replace(0, np.nan)
    next_puts_df = pd.DataFrame.from_dict(next_puts, orient='index').sort_index().replace(0, np.nan)

    return near_calls_df, near_puts_df, next_calls_df, next_puts_df

## XVBT Implementation

Replication of CBOE VIX calculation.

Near and next term expiries are defined as the next two Fridays respectively. Bid/ask data for all strike puts and calls are retrieved from Deribit for these expiries.

In [508]:
api = dbitApi.MarketDataApi()

now = datetime.now()
near_expiry, next_expiry, near_datetime, next_datetime = get_near_next_terms(now)

print(near_expiry, next_expiry)

1MAY20 8MAY20


In [509]:
near_instruments = get_instruments_with_expiry(near_expiry)
next_instruments = get_instruments_with_expiry(next_expiry)

near_calls_df, near_puts_df, next_calls_df, next_puts_df = get_bids_asks(near_instruments, next_instruments)

In [510]:
near_calls_df

,best_bid,best_ask
5250,0.3010,0.3380
5500,0.2675,0.3065
5750,0.2345,0.2750
6000,0.2145,0.2440
6250,0.1830,0.2115
6500,0.1345,0.1620
6750,0.1255,0.1315
7000,0.0950,0.0990
7250,0.0655,0.0680
7500,0.0380,0.0400


### Step 1: Select the options to be used in the VIX Index calculation

Call and put prices are computed as the average of the respective bid and ask prices. The strike at which the call and put price difference is found to calculate forward prices and separation strikes.

In [511]:
near_prices = pd.DataFrame(index=near_calls_df.index)
near_prices['call_price'] = (near_calls_df['best_bid'] + near_calls_df['best_ask']) / 2
near_prices['put_price'] = (near_puts_df['best_bid'] + near_puts_df['best_ask']) / 2
near_prices['abs_diff'] = abs(near_prices['call_price'] - near_prices['put_price'])

min_near_strike = near_prices['abs_diff'].idxmin()
min_near_diff = near_prices.loc[min_near_strike].abs_diff

next_prices = pd.DataFrame(index=next_calls_df.index)
next_prices['call_price'] = (next_calls_df['best_bid'] + next_calls_df['best_ask']) / 2
next_prices['put_price'] = (next_puts_df['best_bid'] + next_puts_df['best_ask']) / 2
next_prices['abs_diff'] = abs(next_prices['call_price'] - next_prices['put_price'])

min_next_strike = next_prices['abs_diff'].idxmin()
min_next_diff = next_prices.loc[min_next_strike].abs_diff

near_prices

,call_price,put_price,abs_diff
5250,0.31950,NaN,NaN
5500,0.28700,NaN,NaN
5750,0.25475,NaN,NaN
6000,0.22925,0.00075,0.22850
6250,0.19725,0.00100,0.19625
6500,0.14825,0.00150,0.14675
6750,0.12850,0.00200,0.12650
7000,0.09700,0.00275,0.09425
7250,0.06675,0.00450,0.06225
7500,0.03900,0.00925,0.02975


The XVBT index is set to have a constant maturity of seven days and a yield rate of zero (which should not make a difference to calculations - refer to Alexander paper page 9). This is used to calculate forward prices f1, f2 and separation strikes k0_1, k0_2.

In [512]:
const_mature_days = 7
R = 0

n1 = (near_datetime - now).total_seconds() / 60
n2 = (next_datetime - now).total_seconds() / 60
nY = 525600
n = const_mature_days * 24 * 60

t1 = n1/nY
t2 = n2/nY

# Compute forward prices and at-the-money strikes
f1 = min_near_strike + np.e**(R*t1) * min_near_diff
k0_1 = max([strike for strike in near_prices.index if strike <= min_near_strike])

f2 = min_next_strike + np.e**(R*t2) * min_next_diff
k0_2 = max([strike for strike in next_prices.index if strike <= min_next_strike])

print(k0_1, f1, k0_2, f2)

7750 7750.00225 7750 7750.003


Out of the money calls and puts are found by using the calculated separation strikes and excluding at the money strikes.

In [513]:
near_otm_puts_df = near_puts_df.loc[:k0_1][:-1]
near_otm_calls_df = near_calls_df.loc[k0_1:][1:]
next_otm_puts_df = next_puts_df.loc[:k0_2][:-1]
next_otm_calls_df = next_calls_df.loc[k0_2:][1:]

In [514]:
near_otm_puts_df

,best_bid,best_ask
5250,NaN,0.0010
5500,NaN,0.0005
5750,NaN,0.0010
6000,0.0005,0.0010
6250,0.0005,0.0015
6500,0.0010,0.0020
6750,0.0015,0.0025
7000,0.0025,0.0030
7250,0.0040,0.0050
7500,0.0085,0.0100


In [515]:
near_otm_calls_df

,best_bid,best_ask
8000,0.0080,0.0095
8250,0.0040,0.0050
8500,0.0025,0.0030
8750,0.0015,0.0025
9000,0.0010,0.0015


Strikes following two consecutive bid prices and strikes with zero bids are excluded.

In [516]:
near_otm_puts_df = near_otm_puts_df.sort_index(ascending=False)
near_otm_puts_df = near_otm_puts_df.assign(zero_bid=lambda df: (df['best_bid'] == 0).astype(int))
near_otm_puts_df['zero_bid_cumsum'] = near_otm_puts_df['zero_bid'].cumsum()
near_otm_puts_df = near_otm_puts_df[(near_otm_puts_df['zero_bid_cumsum'] <= 2) & (near_otm_puts_df['best_bid'] > 0)]

near_otm_puts_df

,best_bid,best_ask,zero_bid,zero_bid_cumsum
7500,0.0085,0.0100,0,0
7250,0.0040,0.0050,0,0
7000,0.0025,0.0030,0,0
6750,0.0015,0.0025,0,0
6500,0.0010,0.0020,0,0
6250,0.0005,0.0015,0,0
6000,0.0005,0.0010,0,0


In [517]:
near_otm_calls_df = near_otm_calls_df.assign(zero_bid=lambda df: (df['best_bid'] == 0).astype(int))
near_otm_calls_df['zero_bid_cumsum'] = near_otm_calls_df['zero_bid'].cumsum()
near_otm_calls_df = near_otm_calls_df[(near_otm_calls_df['zero_bid_cumsum'] <= 2) & (near_otm_calls_df['best_bid'] > 0)]

near_otm_calls_df

,best_bid,best_ask,zero_bid,zero_bid_cumsum
8000,0.0080,0.0095,0,0
8250,0.0040,0.0050,0,0
8500,0.0025,0.0030,0,0
8750,0.0015,0.0025,0,0
9000,0.0010,0.0015,0,0


In [518]:
next_otm_puts_df = next_otm_puts_df.sort_index(ascending=False)
next_otm_puts_df = next_otm_puts_df.assign(zero_bid=lambda df: (df['best_bid'] == 0).astype(int))
next_otm_puts_df['zero_bid_cumsum'] = next_otm_puts_df['zero_bid'].cumsum()
next_otm_puts_df = next_otm_puts_df[(next_otm_puts_df['zero_bid_cumsum'] <= 2) & (next_otm_puts_df['best_bid'] > 0)]

next_otm_calls_df = next_otm_calls_df.assign(zero_bid=lambda df: (df['best_bid'] == 0).astype(int))
next_otm_calls_df['zero_bid_cumsum'] = next_otm_calls_df['zero_bid'].cumsum()
next_otm_calls_df = next_otm_calls_df[(next_otm_calls_df['zero_bid_cumsum'] <= 2) & (next_otm_calls_df['best_bid'] > 0)]

In [519]:
next_otm_puts_df

,best_bid,best_ask,zero_bid,zero_bid_cumsum
7500,0.0275,0.0285,0,0
7250,0.0180,0.0190,0,0
7000,0.0120,0.0135,0,0
6750,0.0080,0.0090,0,0
6500,0.0055,0.0070,0,0
6250,0.0040,0.0050,0,0
6000,0.0030,0.0040,0,0
5750,0.0025,0.0030,0,0
5500,0.0020,0.0025,0,0


In [520]:
next_otm_calls_df

,best_bid,best_ask,zero_bid,zero_bid_cumsum
8000,0.0245,0.0265,0,0
8250,0.0155,0.0175,0,0
8500,0.0105,0.0120,0,0
8750,0.0075,0.0085,0,0
9000,0.0055,0.0065,0,0
9250,0.0040,0.0055,0,0
9500,0.0035,0.0045,0,0


### Step 2: Calculate volatility for both near-term and next-term options

Refer to VIX white paper page 8.

In [521]:
near_calc_strikes_df = pd.DataFrame(index=near_prices.index)
near_calc_strikes_df['price'] = (near_otm_puts_df['best_bid'] + near_otm_puts_df['best_ask']) / 2
near_calc_strikes_df['price'] = near_calc_strikes_df.price.combine_first((near_otm_calls_df['best_bid'] + near_otm_calls_df['best_ask']) / 2)
near_calc_strikes_df.at[k0_1] = (near_prices.loc[k0_1].call_price + near_prices.loc[k0_1].put_price) / 2

In [522]:
near_calc_strikes_df = near_calc_strikes_df.dropna()
near_calc_strikes_df

,price
6000,0.000750
6250,0.001000
6500,0.001500
6750,0.002000
7000,0.002750
7250,0.004500
7500,0.009250
7750,0.020625
8000,0.008750
8250,0.004500


In [523]:
next_calc_strikes_df = pd.DataFrame(index=next_prices.index)
next_calc_strikes_df['price'] = (next_otm_puts_df['best_bid'] + next_otm_puts_df['best_ask']) / 2
next_calc_strikes_df['price'] = next_calc_strikes_df.price.combine_first((next_otm_calls_df['best_bid'] + next_otm_calls_df['best_ask']) / 2)
next_calc_strikes_df.at[k0_2] = (next_prices.loc[k0_2].call_price + next_prices.loc[k0_2].put_price) / 2

In [524]:
next_calc_strikes_df = next_calc_strikes_df.dropna()
next_calc_strikes_df

,price
5500,0.00225
5750,0.00275
6000,0.00350
6250,0.00450
6500,0.00625
6750,0.00850
7000,0.01275
7250,0.01850
7500,0.02800
7750,0.04025


In [525]:
near_sum = 0
for i in range(len(near_calc_strikes_df)):
    row = near_calc_strikes_df.iloc[i]
    if i == 0:
        deltaKi = near_calc_strikes_df.iloc[i+1].name - row.name
    elif i == len(near_calc_strikes_df) - 1:
        deltaKi = row.name - near_calc_strikes_df.iloc[i-1].name
    else:
        deltaKi = (near_calc_strikes_df.iloc[i+1].name - near_calc_strikes_df.iloc[i-1].name) / 2

    near_sum += deltaKi/(row.name ** 2) * np.e**(R*t1) * row.price
    
next_sum = 0
for i in range(len(next_calc_strikes_df)):
    row = next_calc_strikes_df.iloc[i]
    if i == 0:
        deltaKi = next_calc_strikes_df.iloc[i+1].name - row.name
    elif i == len(next_calc_strikes_df) - 1:
        deltaKi = row.name - next_calc_strikes_df.iloc[i-1].name
    else:
        deltaKi = (next_calc_strikes_df.iloc[i+1].name - next_calc_strikes_df.iloc[i-1].name) / 2
    
    next_sum += deltaKi/(row.name ** 2) * np.e**(R*t2) * row.price
    
sigma1 = ((2/t1) * near_sum) - (1/t1)*((f1/k0_1 - 1)**2)
sigma2 = ((2/t2) * next_sum) - (1/t2)*((f2/k0_2 - 1)**2)

print(sigma1, sigma2)

7.318646370231625e-05 6.740175909541954e-05


In [526]:
VXBT = 100 * np.sqrt(((t1*sigma1)*((n2-n)/(n2-n1)) + (t2*sigma2)*((n-n1)/(n2-n1)))*(nY/n))

In [527]:
VXBT

0.8259736519523683

## AVXBT and GVXBT Implementation

Refer to *'The Crypto Investor Fear Gauge and the Bitcoin Variance Risk Premium'* by Carol Alexander and Arben Imeraj.

In [552]:
omega = ((n2-nY)/(n2-n1))*n

GVXBT = np.sqrt(omega*t1*sigma1 + (1-omega)*t2*sigma2)

In [553]:
GVXBT

0.8000488410589683

In [554]:
sigma1_a = sigma1 * (f1**-2)
sigma2_a = sigma2 * (f2**-2)

AVXBT = np.sqrt(omega*t1*sigma1_a + (1-omega)*t2*sigma2_a)

In [555]:
AVXBT

0.00010323206433848177

***

# Test implementation against CBOE VIX for S&P 500 options

In [528]:
from vxbt_calc import vxbt_calc as vc
from datetime import timedelta

<module 'vxbt_calc.vxbt_calc' from '/home/fp361/bitfear/vix-implementation/vxbt_calc/vxbt_calc.py'>

CBOE's VIX takes options expiring between 23 and 37 days from now as near-term and next-term options (see CBOE VIX White Paper). Assume exact time of expiry can be neglected for now.

In [529]:
now = datetime.now().date()

start_date = now + timedelta(days=23)
end_date = now + timedelta(days=37)

fridays = [day for row in calendar.Calendar(firstweekday=calendar.MONDAY).yeardatescalendar(now.year) for month in row for week in month for day in week if day.weekday() == calendar.FRIDAY]
near_exp, next_exp = [friday for friday in fridays if friday > start_date and friday < end_date]

near_exp, next_exp

(datetime.date(2020, 5, 22), datetime.date(2020, 5, 29))

Manually download data CSVs from https://www.barchart.com/stocks/quotes/$SPX/options to process

In [530]:
near_data = pd.read_csv('$spx-options-exp-2020-05-22-show-all-stacked-04-27-2020.csv', skipfooter=1)
next_data = pd.read_csv('$spx-options-exp-2020-05-29-show-all-stacked-04-27-2020.csv', skipfooter=1)

/home/fp361/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/home/fp361/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


In [531]:
near_data

,Symbol,Strike,Last,% From Last,Bid,Midpoint,Ask,Change,%Chg,IV,Volume,Open Int,Type,DTE,Exp Date,Time
0,$SPX|20200522|1000.00WC,"1,000.00",0.00,-65.23%,"1,865.40","1,869.30","1,873.20",0.00,unch,0.00%,0,0,Call,25,05/22/20,NaN
1,$SPX|20200522|1050.00WC,"1,050.00",0.00,-63.49%,"1,815.20","1,819.45","1,823.70",0.00,unch,0.00%,0,0,Call,25,05/22/20,NaN
2,$SPX|20200522|1100.00WC,"1,100.00",0.00,-61.75%,"1,766.20","1,770.10","1,774.00",0.00,unch,0.00%,0,0,Call,25,05/22/20,NaN
3,$SPX|20200522|1150.00WC,"1,150.00",0.00,-60.01%,"1,715.80","1,719.65","1,723.50",0.00,unch,0.00%,0,0,Call,25,05/22/20,NaN
4,$SPX|20200522|1200.00WC,"1,200.00",0.00,-58.27%,"1,665.30","1,669.55","1,673.80",0.00,unch,0.00%,0,0,Call,25,05/22/20,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,$SPX|20200522|500.00WP,500.00,0.00,-82.61%,0.00,0.03,0.05,0.00,unch,0.00%,0,0,Put,25,05/22/20,NaN
444,$SPX|20200522|600.00WP,600.00,0.05,-79.14%,0.00,0.03,0.05,0.05,unch,0.00%,60,60,Put,25,05/22/20,04/22/20
445,$SPX|20200522|700.00WP,700.00,0.00,-75.66%,0.00,0.05,0.10,0.00,unch,0.00%,0,0,Put,25,05/22/20,NaN
446,$SPX|20200522|800.00WP,800.00,0.10,-72.18%,0.05,0.08,0.10,0.00,unch,148.47%,235,474,Put,25,05/22/20,04/22/20


In [532]:
near_calls_df = near_data[near_data['Type'] == 'Call'][['Strike', 'Bid', 'Ask']].replace(',', '', regex=True).astype('float').replace(0, np.nan).set_index('Strike').sort_index().rename({'Bid': 'best_bid', 'Ask': 'best_ask'}, axis=1)
near_puts_df = near_data[near_data['Type'] == 'Put'][['Strike', 'Bid', 'Ask']].replace(',', '', regex=True).astype('float').replace(0, np.nan).set_index('Strike').sort_index().rename({'Bid': 'best_bid', 'Ask': 'best_ask'}, axis=1)

next_calls_df = next_data[next_data['Type'] == 'Call'][['Strike', 'Bid', 'Ask']].replace(',', '', regex=True).astype('float').replace(0, np.nan).set_index('Strike').sort_index().rename({'Bid': 'best_bid', 'Ask': 'best_ask'}, axis=1)
next_puts_df = next_data[next_data['Type'] == 'Put'][['Strike', 'Bid', 'Ask']].replace(',', '', regex=True).astype('float').replace(0, np.nan).set_index('Strike').sort_index().rename({'Bid': 'best_bid', 'Ask': 'best_ask'}, axis=1)

In [533]:
near_calls_df

,best_bid,best_ask
Strike,,
400.0,2465.0,2473.70
500.0,2365.0,2373.50
600.0,2265.2,2273.00
700.0,2166.0,2173.80
800.0,2065.1,2073.60
...,...,...
3800.0,NaN,0.15
3900.0,NaN,0.10
4000.0,NaN,0.10


Set maturity to 30 days as specified in VIX White Paper and arbitrarily use value of R1 from the paper as the yield rate (effect is negligible).

In [535]:
maturity = 30
rate = 0.000305

VIX, _1, _2 = vc.calculate_indices(now, near_exp, next_exp, maturity, rate, near_calls_df, near_puts_df, next_calls_df, next_puts_df)
VIX

33.7362171472354

Get the value of VIX from Yahoo Finance at the time options data was downloaded:

In [2]:
import yfinance as yf

In [12]:
yf_vix = yf.Share('^VIX')

AttributeError: module 'yfinance' has no attribute 'Share'

In [24]:
dir(yf_vix)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_balancesheet',
 '_base_url',
 '_calendar',
 '_cashflow',
 '_download_options',
 '_earnings',
 '_expirations',
 '_financials',
 '_fundamentals',
 '_get_fundamentals',
 '_history',
 '_info',
 '_institutional_holders',
 '_isin',
 '_major_holders',
 '_options2df',
 '_recommendations',
 '_scrape_url',
 '_sustainability',
 'actions',
 'balance_sheet',
 'balancesheet',
 'calendar',
 'cashflow',
 'dividends',
 'earnings',
 'financials',
 'get_actions',
 'get_balance_sheet',
 'get_balancesheet',
 'get_calendar',
 'get_cashflow',
 'get_dividends',
 'get_earnings',
 'get_financials',
 'get_info',
 'get_institutional_holders',
 'ge

In [28]:
yf_vix._get_fundamentals()

### Value matches.

Not an exact match but this is expected as US Treasury yield rates and exact expiry times are neglected in our calculation.

In [18]:
sp = yf.Ticker('^SPX')
near_opts = sp.option_chain('2020-06-26')
near_opts.puts

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,SPXW200626P00500000,2020-05-27 17:40:12,500.0,0.05,0.00,0.00,0.0,0.0,NaN,2,0.500005,False,REGULAR,USD
1,SPXW200626P01050000,2020-05-29 19:50:02,1050.0,0.05,0.05,0.00,0.0,0.0,25.0,25,2.937503,False,REGULAR,USD
2,SPXW200626P01200000,2020-05-22 18:48:04,1200.0,0.16,0.00,0.05,0.0,0.0,5.0,4,2.593754,False,REGULAR,USD
3,SPXW200626P01300000,2020-05-20 14:54:15,1300.0,0.26,0.00,0.05,0.0,0.0,10.0,0,2.382817,False,REGULAR,USD
4,SPXW200626P01350000,2020-06-15 04:05:32,1350.0,0.10,0.00,0.00,0.0,0.0,20.0,20,1.000005,False,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,SPXW200626P04000000,2020-06-09 23:05:59,4000.0,1178.01,848.70,853.00,0.0,0.0,NaN,4,0.000010,True,REGULAR,USD
217,SPXW200626P04100000,2020-06-16 19:38:50,4100.0,982.40,948.70,953.00,0.0,0.0,NaN,1,0.000010,True,REGULAR,USD
218,SPXW200626P04200000,2020-06-15 04:05:38,4200.0,1187.90,1048.70,1053.00,0.0,0.0,NaN,1,0.000010,True,REGULAR,USD
219,SPXW200626P04300000,2020-06-10 19:27:17,4300.0,1100.10,1148.70,1153.00,0.0,0.0,5.0,5,0.000010,True,REGULAR,USD


In [22]:
near_calls_df = near_opts.calls[['strike', 'bid', 'ask']].replace(',', '', regex=True).astype('float').replace(0, np.nan).set_index('strike').sort_index().rename({'bid': 'best_bid', 'ask': 'best_ask'}, axis=1)
near_calls_df

,best_bid,best_ask
strike,,
900.0,2246.9,2251.2
1100.0,2046.9,2051.2
1200.0,1946.7,1950.9
1300.0,1846.7,1850.9
1400.0,1746.9,1751.2
...,...,...
3650.0,NaN,NaN
3700.0,NaN,NaN
3800.0,NaN,NaN
